# Python Basics for Neuro/DataSci
## Part 2: Advanced Concepts & Event-Related Potentials (ERP)

Welcome to Part 2! In Part 1, you learned the fundamental Python concepts. Now we'll build on those skills with advanced techniques and apply them to real neuroscience data analysis.

### What We'll Cover:
1. **Advanced Data Handling**: Multi-dimensional arrays and trial-based data
2. **Statistical Concepts**: Trial averaging, confidence intervals, and variability
3. **Real EEG Analysis**: Event-Related Potentials from human brain recordings
4. **Professional Methods**: Bootstrap statistics and hypothesis testing
5. **Research Workflow**: Complete neuroscience data analysis pipeline

### Prerequisites Met in Part 1:
- Python syntax and data types
- NumPy arrays, indexing, and basic operations
- Matplotlib visualization
- Functions and control flow
- File loading basics

Let's bridge from Part 1 concepts to advanced neuroscience analysis!

# Bridging Part 1 to Advanced Analysis

Great job completing Part 1! Before we dive into real brain data, let's build some advanced concepts that bridge your foundational skills to professional neuroscience analysis.

## Why These Advanced Concepts Matter

In Part 1, you worked mostly with simple arrays and individual signals. Real neuroscience data is more complex:
- **Multiple trials** of the same experiment (to reduce noise)
- **Multi-dimensional datasets** (trials × time points × channels)
- **Statistical uncertainty** (how reliable are our measurements?)
- **Hypothesis testing** (are differences between conditions real?)

Let's build these concepts step by step, using the same tools you already know from Part 1!

## 1. Working with Multi-Dimensional Data

Many neuroscience datasets have multiple dimensions. Understanding how to handle these is crucial for analyzing real brain recordings.

In [ ]:
# Import the libraries we learned in Part 1
import numpy as np
import matplotlib.pyplot as plt

# Create a multi-trial dataset (like real neuroscience experiments!)
np.random.seed(123)  # For reproducibility
n_trials = 200 # 1000 trials
n_timepoints = 500 # 500 samples
time = np.linspace(0, 1, n_timepoints) # 500 time points (samples) from 0 to 1 second

# Simulate multiple trials of a signal with noise
# This mimics how real experiments work: same stimulus, different noise each trial
trials_data = np.zeros((n_trials, n_timepoints))
for trial in range(n_trials):
    # Each trial has the same underlying signal plus different noise
    signal = np.sin(2 * np.pi * 5 * time)  # 5 Hz signal (like a brain rhythm)
    noise = np.random.normal(0, 0.5, n_timepoints)  # Random noise
    trials_data[trial] = signal + noise

print(f"Data shape: {trials_data.shape}")
print("This means: {} trials × {} time points".format(*trials_data.shape))
print("\nThis is exactly like real EEG data structure!")

# Complicated plot: it's just ploting trials 1-12, 13-999, and the last trial to not overcrowd the legend
plt.figure(figsize=(10, 6))
for i in range(n_trials):
    if i < 9:
        plt.plot(time, trials_data[i], alpha=0.3, label=f'Trial {i+1}')
    elif i == 9:
        plt.plot(time, trials_data[i], alpha=0.3, label='Trial 10')
    elif i == 10:
        plt.plot(time, trials_data[i], alpha=0.3, label='Trial 11')
    elif i == 11:
        plt.plot(time, trials_data[i], alpha=0.3, label='Trial 12')
    elif i == 99:
        plt.plot(time, trials_data[i], alpha=0.3, label='Trial 13-999')
    elif i == n_trials - 1:
        plt.plot(time, trials_data[i], alpha=0.3, label=f'Trial {n_trials}')
    else:
        plt.plot(time, trials_data[i], alpha=0.3)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Multiple Trials: Same Signal + Different Noise')
plt.legend()
plt.grid(True)
plt.show()


# Simpler plot: all trials overlayed but only label first 10 trials to avoid overcrowding the legend
plt.figure(figsize=(10,6))
for i in range(n_trials):
    plt.plot(time, trials_data[i], alpha=0.1, label= f'Trial {i+1}' if i < 10 else None)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('All Trials Overlayed')
plt.legend()
plt.grid(True)
print("Notice: Each trial looks different due to noise,")
print("but they all contain the same underlying 5 Hz signal!")
plt.show()

## 2a. The Power of Averaging: From Noise to Signal

Here's the key insight that makes neuroscience possible: **averaging across trials reveals hidden signals**!

This is the foundation of Event-Related Potentials (ERPs) and many other neuroscience techniques:
- Individual trials are noisy (hard to see the signal)
- Random noise cancels out when averaged
- Consistent signals become clear
- This is why we need many trials in experiments

Let's see this magic in action:

In [ ]:
# Compute the average across trials (this is what an ERP is!)
average_signal = np.mean(trials_data, axis=0)  # axis=0 means average across trials

# Create a comparison plot: Individual trials vs Average
plt.figure(figsize=(12, 8))

# Top subplot: Individual trials (noisy)
plt.subplot(2, 1, 1)
for i in range(n_trials):
    plt.plot(time, trials_data[i], 'gray', alpha=0.3)
plt.plot(time, trials_data[0], 'blue', alpha=0.7, linewidth=2,
          label='One single trial out of 10')
plt.ylabel('Amplitude')
plt.title('Individual Trials (Noisy - hard to see the signal!)')
plt.legend()
plt.grid(True)

# Bottom subplot: Average (clean!)
plt.subplot(2, 1, 2)
plt.plot(time, average_signal, 'red', linewidth=3, 
         label='Average across all trials')
plt.plot(time, np.sin(2 * np.pi * 5 * time), 'black', linestyle='--', 
         linewidth=2, label='True underlying signal')
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Average Signal (Clean - the signal emerges!)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print("The Magic of Averaging!")
print(f"Single trial amplitude variation: ±{np.std(trials_data[0]):.3f}")
print(f"Average signal peak: {np.max(np.abs(average_signal)):.3f}")
print(f"True signal peak: 1.000")
print(f"Accuracy improvement: {((1.0 - abs(np.max(np.abs(average_signal)) - 1.0))/1.0)*100:.1f}%")
print("\nThis is EXACTLY what we'll do with real brain signals!")
print("EEG trials are noisy → average them → reveal the ERP!")

## 2b. Statistical Thinking: Uncertainty and Confidence

When we compute an average, we need to know: **How reliable is this measurement?**

We can use **confidence intervals** to quantify uncertainty:
- **Standard deviation**: How much do individual trials vary?
- **Standard error**: How much might our average vary if we repeated the experiment?
- **Confidence intervals**: The range where the true value likely lies

This statistical thinking is crucial for interpreting brain data!

In [ ]:
# Calculate statistical measures (using NumPy functions from Part 1!)
signal_std = np.std(trials_data, axis=0)  # Standard deviation across trials
signal_sem = signal_std / np.sqrt(n_trials)  # Standard error of the mean

# 95% confidence intervals (±2 standard errors)
ci_upper = average_signal + 2 * signal_sem
ci_lower = average_signal - 2 * signal_sem

# Create a professional statistical plot
plt.figure(figsize=(10, 6))
plt.plot(time, average_signal, 'red', linewidth=3, label='Average signal')
plt.fill_between(time, ci_lower, ci_upper, 
                 alpha=0.3, color='red', label='95% Confidence Interval')
plt.plot(time, np.sin(2 * np.pi * 5 * time), 'black', linestyle='--', 
         linewidth=2, label='True signal', alpha=0.7)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.title('Average Signal with Statistical Confidence')
plt.legend()
plt.grid(True)
plt.show()

print("Statistical Summary:")
print(f"Number of trials: {n_trials}")
print(f"Average amplitude: {np.max(average_signal):.3f}")
print(f"Standard error: {np.max(signal_sem):.3f}")
print(f"Confidence interval width: {np.max(ci_upper - ci_lower):.3f}")
print("\nInterpretation:")
print("- Confidence interval shows measurement reliability")
print("- Narrower intervals = more confident in our measurement")
print("- This helps us distinguish real signals from noise")
print("\nNow we're ready for real brain data analysis!")

## 3. Making Connections to Real Brain Data Analysis

Perfect! You now understand the advanced concepts needed for professional neuroscience analysis:
- Multi-dimensional data handling
- Trial averaging for noise reduction  
- Statistical confidence intervals
- Professional visualization techniques

Now let's apply these **exact same concepts** to real human brain recordings!

### The EEG Experiment

**Scenario**: An undergraduate student listens to two types of audio tones while we record brain activity.

**Data**: 1000 trials each of high-pitch and low-pitch tones, recorded at 500 Hz for 1 second per trial (sampling rate 500 Hz, i.e., 500 EEG electrode readings/samples per second),

**Question**: Do the brain responses differ between the two tone types?

**Method**: Use everything we just learned - the same code, just with real brain data!

### Before We Compute ERPs
So far we've:
1. Seen how multiple noisy trials conceal an underlying signal.
2. Watched averaging reveal that signal.
3. Quantified uncertainty with confidence intervals.

In the real EEG data we are about to:
- Load trials from two experimental conditions.
- Visualize a few single trials (to appreciate the noise level).
- Align them in time to the stimulus.
- Compute the ERP for each condition (just a mean across trials!).
- Attach 95% confidence information to those ERPs.
- Then compare the two conditions.

> Key idea: An ERP is **nothing more** than the trial-wise average, interpreted in the context of timing and uncertainty.

Next, we'll start by looking at a couple of individual real EEG trials, and then move directly into computing the ERPs in Section 4b.

In [ ]:
# Load real brain data using the same libraries from Part 1!
from scipy.io import loadmat

# Load the EEG data (just like loading any data file)
data = loadmat('02_EEG-1.mat')
EEGa = data['EEGa']             # EEG data from condition A (high pitch tone)
EEGb = data['EEGb']             # EEG data from condition B (low pitch tone)
t = data['t'][0]                # Time axis

# Examine the data structure (using skills we just practiced!)
ntrials, nsamples = EEGa.shape  # Multi-dimensional data - just like our simulation!
print(f"Real EEG data shape: {ntrials} trials × {nsamples} time points")
print("Same structure as our simulated data, but now it's real brain signals!")

print(f"\nExperiment details:")
print(f"Trial duration: {t[-1]:.3f} seconds")
print(f"Sampling rate: {1/(t[1]-t[0]):.0f} Hz") 
print(f"Stimulus occurs at: 0.25 seconds")
print(f"Total trials: {ntrials} per condition")

# Compare data ranges between conditions
print(f"\nData characteristics:")
print(f"Condition A (high pitch) voltage range: {np.ptp(EEGa):.2f} µV")
print(f"Condition B (low pitch) voltage range: {np.ptp(EEGb):.2f} µV")

print("\nThis is real human brain activity!")

In [ ]:
# Plot single trial from condition A
plt.figure(figsize=(12, 4))
plt.plot(t, EEGa[0])                     # Plot condition A, trial 1 data vs t.
plt.xlabel('Time [s]')                   # Label the x-axis as time.
plt.ylabel('Voltage [µV]')               # Label the y-axis as voltage.
plt.title('EEG data from condition A, Trial 1')  # Add a title

# Add a vertical line to indicate the stimulus time
plt.axvline(x=0.25, color='red', linestyle='--', linewidth=2, label='Stimulus onset')
plt.legend()
plt.grid(True)
plt.show()

# Let's also examine the first trial from condition B for comparison
plt.figure(figsize=(12, 4))
plt.plot(t, EEGa[0], label='Condition A (High pitch)', alpha=0.7)
plt.plot(t, EEGb[0], 'r', label='Condition B (Low pitch)', alpha=0.7)
plt.xlabel('Time [s]')
plt.ylabel('Voltage [µV]')
plt.title('Single trial comparison: Condition A vs Condition B')
plt.axvline(x=0.25, color='black', linestyle='--', linewidth=2, label='Stimulus onset')
plt.legend()
plt.grid(True)
plt.show()

print("Notice how noisy the single trial data is!")
print("This is why we need to average across many trials to see the ERP.")

## 4a. Computing Event-Related Potentials (ERPs)

In Section 4 we framed the problem and reviewed why averaging reveals stimulus-locked brain responses. You've now seen the raw single-trial variability — so this step is simply turning those many noisy repetitions into a clearer signal.

### What We Do Here
1. Average EEG signals across trials (one mean waveform per condition)
2. Align interpretation to the stimulus onset
3. Quantify uncertainty with a 95% confidence interval (via the Central Limit Theorem approximation: mean ± 2·SEM)
4. Prepare for statistical comparison between conditions

> ***Background**: The Central Limit Theorem states that the mean of a sufficiently large number of independent random variables, each with finite mean and variance, will be approximately normally distributed

> ***Question**: What assumptions do we make by using the central limit theorem?

> **Answer**: We assume that 1) the data is independent across trials, and 2) that the data's variance and mean is always finite (does not go to plus or minus infinity)

> ***Question**: Are these reasonable assumptions to make?

> **Answer**: Assumption 1: Yes, but may not be for some other experiments where one trial's data may be dependent on (aka influenced by) the previous trial (for example, an experiement where a participant may be learning or memorizing information from previous trials) Assumption 2: Yes because we can assume that brains EEG recordings don't go to plus or minus infinity, and so the variance and mean can be assumed to remain finite.



In [ ]:
# Compute ERPs by averaging across trials
mnA = np.mean(EEGa, axis=0)  # ERP for condition A
mnB = np.mean(EEGb, axis=0)  # ERP for condition B

# Compute standard deviation
sdA = np.std(EEGa, axis=0)   # Std across trials (A)
sdB = np.std(EEGb, axis=0)   # Std across trials (B)

# Standard error of the mean
semnA = sdA / np.sqrt(ntrials)
semnB = sdB / np.sqrt(ntrials)

# 95% CI bounds (using CLT: ±2*SEM)
upperA = mnA + 2 * semnA
lowerA = mnA - 2 * semnA
upperB = mnB + 2 * semnB
lowerB = mnB - 2 * semnB


plt.figure(figsize=(12, 6))

# Condition A (mean->upper and mean->lower)
plt.fill_between(t, mnA, upperA, where=upperA>=mnA, color='blue', alpha=0.15, linewidth=0)
plt.fill_between(t, mnA, lowerA, where=lowerA<=mnA, color='blue', alpha=0.08, linewidth=0)

# Condition B (mean->upper and mean->lower)
plt.fill_between(t, mnB, upperB, where=upperB>=mnB, color='red', alpha=0.15, linewidth=0)
plt.fill_between(t, mnB, lowerB, where=lowerB<=mnB, color='red', alpha=0.08, linewidth=0)

# Mean ERPs
plt.plot(t, mnA, 'b-', linewidth=2.5, label='Condition A (High pitch)')
plt.plot(t, mnB, 'r-', linewidth=2.5, label='Condition B (Low pitch)')


# Stimulus timing
plt.axvline(x=0.25, color='black', linestyle='--', linewidth=2, label='Stimulus onset')
plt.axhline(0, t[0], t[-1], color='gray', linewidth=0.8, alpha=0.7)

plt.xlabel('Time [s]')
plt.ylabel('Voltage [µV]')
plt.title('Event-Related Potentials with Mean-to-CI Shading (95% CI)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print some basic statistics
print("ERP Analysis Results:")
print(f"Number of trials per condition: {ntrials}")
print("\nCondition A ERP:")
print(f"  Peak amplitude: {np.max(np.abs(mnA)):.3f} µV")
print(f"  Time of peak: {t[np.argmax(np.abs(mnA))]:.3f} s")
print("\nCondition B ERP:")
print(f"  Peak amplitude: {np.max(np.abs(mnB)):.3f} µV")
print(f"  Time of peak: {t[np.argmax(np.abs(mnB))]:.3f} s")

## 4b. Comparing ERPs Between Conditions (without Bootstrap)

Now that we have computed ERPs for both conditions, let's examine their differences more carefully. We'll compute the difference wave and assess whether the conditions significantly differ.

If zero isn't included within the range of the confidence intervals, we can say that the conditions significantly differ.

In [ ]:
# Compute the difference between mean signals across ERP conditions
mnA = np.mean(EEGa,0)          # Determine ERP for condition A
mnB = np.mean(EEGb,0)          # Determine ERP for condition B
sdmeanA = np.std(EEGa,0) / np.sqrt(ntrials) # Compute standard deviation of the mean for condition A
sdmeanB = np.std(EEGb,0) / np.sqrt(ntrials) # Compute standard deviation of the meanfor condition B
mnD = mnA - mnB             # Compute the mean difference between ERPs
stat = np.max(np.abs(mnD))        # Compute the statistic (max absolute difference)
print('stat = {:.4f}'.format(stat))

# Standard deviation of the mean difference
sdmeanD = np.sqrt(sdmeanA**2 + sdmeanB**2) 

# Plot the difference wave with confidence intervals
plt.figure(figsize=(12, 5))

# Plot difference wave
plt.plot(t, mnD, 'k-', linewidth=3, label='Difference (A - B)')
plt.plot(t, mnD + 2 * sdmeanD, 'k:', linewidth=1, alpha=0.7, label='95% CI')
plt.plot(t, mnD - 2 * sdmeanD, 'k:', linewidth=1, alpha=0.7)

# Add reference lines
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
plt.axvline(x=0.25, color='red', linestyle='--', linewidth=2, label='Stimulus onset')

plt.xlabel('Time [s]')
plt.ylabel('Voltage Difference [µV]')
plt.title('Difference Wave: Condition A - Condition B')
plt.legend()
plt.grid(True)
plt.show()


## 5a. Doing it Again Differently: Bootstrap Confidence Intervals for ERPs

While the Central Limit Theorem gives us a way to compute confidence intervals, we can also use bootstrapping; a powerful computational technique that doesn't rely on assumptions about the underlying distribution.

**Bootstrap procedure:**
The bootstrap procedure allows us to estimate the distribution of many different statistics, offering more versatility. This allows us to estimate the confidence intervals without relying on the assumption that the underlying distribution is approximately normal.

**How we'll implement this:**
1. Sample with replacement 1,000 trials of the EEG data from condition A, then B.
2. Compute the ERP for each resampled dataset (again, just averaging the trials)
3. Repeat many times (3,000 times) to create a distribution of possible ERPs
4. Use percentiles of this distribution to construct confidence intervals

Let's implement bootstrap confidence intervals for our ERPs:

In [ ]:
# Bootstrap function for computing bootstrap confidence intervals for both ERPs (overlaid plot)
import numpy as np
import matplotlib.pyplot as plt

def bootstrapERPdata(EEGdata, size=None):
    """Return one bootstrap ERP (mean of resampled trials)."""
    ntrials_local = len(EEGdata)
    if size is None:
        size = ntrials_local
    idx = np.random.randint(ntrials_local, size=size)
    return EEGdata[idx].mean(0)

n_bootstrap = 3000
print(f"Bootstrapping {n_bootstrap} ERPs per condition for 95% CIs...")

# Generate bootstrap samples
bootstrapERPA = np.array([bootstrapERPdata(EEGa) for _ in range(n_bootstrap)])
bootstrapERPB = np.array([bootstrapERPdata(EEGb) for _ in range(n_bootstrap)])

# Sort each column for percentile lookup
bootstrapERPA.sort(axis=0)
bootstrapERPB.sort(axis=0)
N = n_bootstrap

# Percentile indices (simple integer lookup)
ciLA = bootstrapERPA[int(0.025 * N)]
ciUA = bootstrapERPA[int(0.975 * N)]
ciLB = bootstrapERPB[int(0.025 * N)]
ciUB = bootstrapERPB[int(0.975 * N)]

# Mean ERPs
mnA = np.mean(EEGa, 0)
mnB = np.mean(EEGb, 0)

# Expose for later summary cell (keep naming convention)
ci_lower_A, ci_upper_A = ciLA, ciUA
ci_lower_B, ci_upper_B = ciLB, ciUB

# Single overlaid plot
plt.figure(figsize=(12, 8))

# Condition A shading & mean
plt.fill_between(t, ci_lower_A, ci_upper_A, color='teal', alpha=0.70, linewidth=0, label='A 95% CI (bootstrap)')
plt.plot(t, mnA, 'b-', lw=2.5, label='Condition A ERP')

# Condition B shading & mean
plt.fill_between(t, ci_lower_B, ci_upper_B, color='pink', alpha=0.70, linewidth=0, label='B 95% CI (bootstrap)')
plt.plot(t, mnB, 'r-', lw=2.5, label='Condition B ERP')

# Reference lines
plt.axvline(0.25, color='black', ls='--', lw=2, label='Stimulus onset')
plt.axhline(0, color='gray', lw=0.8, alpha=0.6)

plt.xlabel('Time [s]')
plt.ylabel('Voltage [µV]')
plt.title('Overlaid ERPs with Bootstrap 95% Confidence Intervals')
plt.legend(frameon=False)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 5b. Comparing the ERPs between both conditions: Bootstrap test

Is this difference significant, or is it due to randomness?

We'll use the bootstrap procedure, similar to how we used it to construct confidence intervals.

This time, we'll merge the two EEG datasets from both conditions (for a total of 2000 trials) and resample with replacement.

> Question: Doesn't that defeat the purpose of having separate conditions?

> Answer: While it may seem counter intuitive to mix up trials from both conditions, if the data from both conditions don't significantly differ, then our test statistic wouldn't significantly differ. This allows us to confirm or deny if these two conditions differ.

We use the null hypothesis h0: there is no significant difference between the two conditions

If the null hypothesis holds, then we would see that the test statistic we calculated above would fall well within the distribution of test statistics from the bootstrap resampling 

**Bootstrap test procedure:**
1. Merge the 1,000 trials each of EEG data from conditions A and B to form a combined distribution of 2,000 trials.
2. Randomly sample with replacement 1,000 trials of EEG data from the combined distribution, and compute the resampled ERP.
3. Repeat step 2 and compute a second resampled ERP.
4. Compute the statistic, the maximum absolute value of the difference between the two resampled ERPs.
5. Repeat steps 2-4, 3,000 times to create a distribution of statistic values.
6. Compare the observed statistic (the one we initially got during 4b) to this distribution of statistic values. If the observed statistic is greater than 95% of the bootstrapped values, then reject the null hypothesis that the two conditions are the same.


In [ ]:
# Bootstrap hypothesis test (6-step procedure) using provided bootstrapERP helper
from numpy.random import randint

# Observed statistic (already computed earlier in cell 4b as stat = np.max(np.abs(mnD)) ... mnD = mean difference = mnA - mnB)
observed_stat = np.max(np.abs(mnD))
print(f"Observed statistic (max |A-B|): {observed_stat:.5f}")

# Merge trials from both conditions under H0
eeg_combined = np.vstack((EEGa, EEGb))
combined_n = eeg_combined.shape[0]
print(f"Combined trials: {combined_n}")

n_bootstrap_test = 3000
sample_size = ntrials
null_stats_max_absolute_difference = np.empty(n_bootstrap_test)

for b in range(n_bootstrap_test):
    # Draw two independent bootstrapped ERPs from pooled data
    erp1 = bootstrapERPdata(eeg_combined, size=sample_size)
    erp2 = bootstrapERPdata(eeg_combined, size=sample_size)
    # add the max absolute difference observed in this bootstrap iteration to null_stats...
    null_stats_max_absolute_difference[b] = np.max(np.abs(erp1 - erp2))
    if b % 500 == 0:
        print(f"  Bootstrap test iteration {b}/{n_bootstrap_test}")

# calculate p-value: average of null stats >= observed stat / proportion of null stats exceeding observed stat
p_value = np.mean(null_stats_max_absolute_difference >= observed_stat)
threshold_95 = np.percentile(null_stats_max_absolute_difference, 95)

print("\nBootstrap Hypothesis Test Results (Resample-with-Replacement Null)")
print("--------------------------------------------------------------")
print(f"Observed statistic:        {observed_stat:.5f}")
print(f"95th percentile threshold: {threshold_95:.5f}")
print(f"P-value (>= observed):     {p_value:.6f}")
if observed_stat > threshold_95:
    print("Decision: REJECT H0 (difference exceeds 95% of null statistics)")
else:
    print("Decision: FAIL TO REJECT H0 (difference within null expectations)")

plt.figure(figsize=(10,6))
plt.hist(null_stats_max_absolute_difference, bins=50, color='lightsteelblue', alpha=0.8, label='Null stats (bootstrap)')
plt.axvline(observed_stat, color='red', linestyle='--', linewidth=2, label=f'Observed = {observed_stat:.4f}')
plt.axvline(threshold_95, color='black', linestyle='-', linewidth=2, label=f'95th pct = {threshold_95:.4f}')
plt.xlabel('Max |ERP1 - ERP2| (null resamples)')
plt.ylabel('Frequency')
plt.title('Bootstrap Null Distribution of Max Absolute ERP Difference')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Create a comprehensive summary figure showing all our ERP analyses
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Top panel: Individual trials and ERPs
axes[0].plot(t, EEGa[0], 'b-', alpha=0.3, label='Single trial A')
axes[0].plot(t, EEGb[0], 'r-', alpha=0.3, label='Single trial B')
axes[0].plot(t, mnA, 'b-', linewidth=3, label='ERP A (avg of 1000 trials)')
axes[0].plot(t, mnB, 'r-', linewidth=3, label='ERP B (avg of 1000 trials)')
axes[0].axvline(x=0.25, color='black', linestyle='--', alpha=0.7, label='Stimulus')
axes[0].set_ylabel('Voltage [µV]')
axes[0].set_title('Single Trials vs ERPs')
axes[0].legend()
axes[0].grid(True)

# Middle panel: ERPs with confidence intervals
axes[1].plot(t, mnA, 'b-', linewidth=3, label='Condition A')
axes[1].fill_between(t, ci_lower_A, ci_upper_A, color='blue', alpha=0.2)
axes[1].plot(t, mnB, 'r-', linewidth=3, label='Condition B') 
axes[1].fill_between(t, ci_lower_B, ci_upper_B, color='red', alpha=0.2)
axes[1].axvline(x=0.25, color='black', linestyle='--', alpha=0.7, label='Stimulus')
axes[1].set_ylabel('Voltage [µV]')
axes[1].set_title('ERPs with Bootstrap 95% Confidence Intervals')
axes[1].legend()
axes[1].grid(True)

# Bottom panel: Difference wave
axes[2].plot(t, mnD, 'k-', linewidth=3, label='Difference (A - B)')
axes[2].plot(t, mnD + 2 * semnD, 'k:', alpha=0.7, label='95% CI (CLT)')
axes[2].plot(t, mnD - 2 * semnD, 'k:', alpha=0.7)
axes[2].axhline(y=0, color='gray', linestyle='-', alpha=0.5)
axes[2].axvline(x=0.25, color='black', linestyle='--', alpha=0.7, label='Stimulus')
axes[2].set_xlabel('Time [s]')
axes[2].set_ylabel('Voltage Difference [µV]')
axes[2].set_title('Difference Wave with Statistical Significance')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

print("ERP Analysis Summary:")
print("="*50)
print(f"Data: {ntrials} trials per condition, {nsamples} time points")
print(f"Sampling rate: {1/(t[1]-t[0]):.0f} Hz")
print(f"Trial duration: {t[-1]:.3f} seconds")
print(f"Stimulus onset: 0.250 seconds")
print()
print("Statistical Results:")
print(f"Bootstrap permutation test p-value: {p_value:.6f}")
print(f"Significant difference: {'YES' if p_value < 0.05 else 'NO'} (α = 0.05)")
print()
print("Peak Analysis:")
print(f"Condition A peak: {np.max(np.abs(mnA)):.3f} µV at {t[np.argmax(np.abs(mnA))]:.3f} s")
print(f"Condition B peak: {np.max(np.abs(mnB)):.3f} µV at {t[np.argmax(np.abs(mnB))]:.3f} s")
print(f"Max difference: {np.max(np.abs(mnD)):.3f} µV at {t[np.argmax(np.abs(mnD))]:.3f} s")

## Summary

In this notebook, we conducted a complete analysis of EEG Event-Related Potentials (ERPs), demonstrating how Python can be used for real neuroscience research. Here's what we accomplished:

### Data Analysis Pipeline
1. **Data Loading**: Imported EEG data from MATLAB format using `scipy.io.loadmat`
2. **Visual Inspection**: Examined single trials to understand the noisy nature of raw EEG
3. **ERP Computation**: Averaged across trials to reveal stimulus-locked neural responses
4. **Statistical Analysis**: Applied both Central Limit Theorem and bootstrap methods for confidence intervals
5. **Hypothesis Testing**: Used bootstrap permutation tests to compare conditions statistically

### Key Python Concepts Applied
- **NumPy arrays** for efficient numerical computation
- **Matplotlib** for comprehensive data visualization  
- **SciPy** for loading scientific data formats
- **Functions** to organize reusable analysis code
- **Control flow** for iterative bootstrap procedures
- **Statistical thinking** through confidence intervals and hypothesis testing

### Scientific Results
We analyzed EEG responses to high-pitch vs low-pitch tones and found:
- Clear event-related potentials in both conditions
- Statistical methods revealed significant differences between conditions
- Bootstrap procedures confirmed our findings without relying on distributional assumptions

### Methodological Insights
- **Single trials are noisy** - averaging across trials is essential to reveal ERPs
- **Multiple statistical approaches** - CLT and bootstrap methods can complement each other
- **Visualization is critical** - plots help us understand both data and statistical results
- **Computational statistics** - bootstrap methods provide powerful, assumption-free analysis tools

## Next Steps in ERP Analysis
- **Topographic analysis**: Examine spatial patterns across multiple electrodes
- **Time-frequency analysis**: Investigate oscillatory components of the response
- **Component analysis**: Use techniques like ICA to separate neural sources
- **Statistical power**: Determine optimal sample sizes for future experiments

## Donate
If you enjoy these tutorials and would like to support continued development of educational neuroscience content, consider supporting the original Case Studies in Neural Data Analysis project at: https://www.paypal.com/donate/?hosted_button_id=DL8P5ZGS9962U

---

**Remember**: The best way to learn is by doing. Take these ERP analysis techniques and apply them to your own research questions!